In [2]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


ValueError: Could not load model distilbert-base-uncased-finetuned-sst-2-english with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForSequenceClassification'>, <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>, <class 'transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification'>).

In [6]:
import nltk
from nltk.corpus import movie_reviews
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Download the movie reviews corpus
nltk.download('movie_reviews')

# Instantiate the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Get the movie review texts and labels
reviews = []
for fileid in movie_reviews.fileids():
    label = fileid.split('/')[0]
    text = movie_reviews.raw(fileid)
    reviews.append((text, label))

# Tokenize the reviews and convert them to input IDs
inputs = []
labels = []
for review in reviews:
    text = review[0]
    label = review[1]
    encoded_dict = tokenizer.encode_plus(text, add_special_tokens=True, max_length=256, pad_to_max_length=True, return_attention_mask=True)
    input_ids = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    inputs.append({'input_ids': torch.tensor(input_ids), 'attention_mask': torch.tensor(attention_mask)})
    labels.append(int(label == 'pos'))

# Create a DataLoader to batch the inputs and labels
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(torch.stack([input['input_ids'] for input in inputs]), torch.stack([input['attention_mask'] for input in inputs]), torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=8)

# Set the model to evaluation mode
model.eval()

# Evaluate the model on the test set
total_accuracy = 0
for batch in dataloader:
    with torch.no_grad():
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        accuracy = (predictions == batch[2]).float().mean()
        total_accuracy += accuracy.item()
print(f"Accuracy: {total_accuracy/len(dataloader):.4f}")


[nltk_data] Downloading package movie_reviews to C:\Users\Noel
[nltk_data]     Nathan\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequen

Accuracy: 0.5000
